In [17]:
#requires NLTK, BeautifulSoup
import load_data

import glob, os
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Embedding, SimpleRNN, Dropout
from keras.preprocessing.text import Tokenizer, one_hot
from keras.preprocessing import sequence
from keras.utils import to_categorical

labels = {"Obama" : 0, "Lincoln": 1, "Trump": 2}
input_text = []
input_labels = []
directory = "../data/processed/"

for filename in glob.glob(os.path.join(directory, '*.txt')):
    arr = filename.replace(directory,'').split("_")
    input_labels = input_labels + [labels[arr[0]]]
    input_text = input_text + [open(filename).read().decode("UTF-8").encode("ascii","ignore")] 
        
tokenizer = Tokenizer(num_words=1000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
               lower=True, split=" ", char_level=False)
tokenizer.fit_on_texts(input_text)
tk_test = tokenizer.texts_to_sequences(input_text)

max_len = 1500
max_features = 20000

x = sequence.pad_sequences(tk_test, maxlen=max_len)
y = to_categorical(input_labels)

model = Sequential()
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 100, input_length=max_len))
model.add(Dropout(0.2))
model.add(SimpleRNN(100))
model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='SGD',metrics=['categorical_accuracy'])
print(model.summary())

model.fit(x, y=y, batch_size=50, nb_epoch=10, verbose=1, validation_split=0.2, shuffle=True)

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 1500, 100)         2000000   
_________________________________________________________________
dropout_9 (Dropout)          (None, 1500, 100)         0         
_________________________________________________________________
simple_rnn_12 (SimpleRNN)    (None, 100)               20100     
_________________________________________________________________
dense_13 (Dense)             (None, 3)                 303       
_________________________________________________________________
activation_13 (Activation)   (None, 3)                 0         
Total params: 2,020,403
Trainable params: 2,020,403
Non-trainable params: 0
_________________________________________________________________
None
Train on 34 samples, validate on 9 samples
Epoch 1/10
1s - loss: 1.1026 - categorical_accuracy: 0.3824 - val_los